# Update IPCC AR6 ERF timeseries

- with updated data to 2022
- using AR6 assessments for components not directly observed

In [ ]:
import json

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import scipy.stats
from tqdm.auto import tqdm
from fair.forcing.ghg import meinshausen2020

In [ ]:
# probablistic ensemble
SAMPLES = 200000
forcing = {}

In [ ]:
NINETY_TO_ONESIGMA = scipy.stats.norm.ppf(0.95)
NINETY_TO_ONESIGMA

In [ ]:
with open('../data/random_seeds.json', 'r') as filehandle:
    SEEDS = json.load(filehandle)

In [ ]:
emissions = pd.read_csv('../output/slcf_emissions_1750-2022.csv', index_col=0)
emissions

In [ ]:
concentrations = pd.read_csv('../output/ghg_concentrations_1750-2022.csv', index_col=0)
for year in range(1751, 1850):
    concentrations.loc[year, :] = np.nan
concentrations.sort_index(inplace=True)
concentrations.interpolate(inplace=True)

In [ ]:
concentrations

In [ ]:
# uncertainties from IPCC
uncertainty_seed = 38572

unc_ranges = np.array([
    0.12,      # CO2
    0.20,      # CH4: updated value from etminan 2016
    0.14,      # N2O
    0.19,      # other WMGHGs
    0.50,      # Total ozone
    1.00,      # stratospheric WV from CH4
    0.70,      # contrails approx - half-normal
    1.25,      # bc on snow - half-normal
    0.50,      # land use change
    5.0/20.0,  # volcanic
    0.50,      # solar (amplitude)
])/NINETY_TO_ONESIGMA

scale = scipy.stats.norm.rvs(
    size=(SAMPLES,11), 
    loc=np.ones((SAMPLES,11)), 
    scale=np.ones((SAMPLES, 11)) * unc_ranges[None,:], 
    random_state=uncertainty_seed
)

## BC snow is asymmetric Gaussian. We scale the half of the distribution above/below best estimate
scale[scale[:,7]<1,7] = 0.08/0.1*(scale[scale[:,7]<1,7]-1) + 1

## Contrails also asymmetric but benefits of scaling are tiny
scale[scale[:,6]<1,6] = 0.0384/0.0406*(scale[scale[:,6]<1,6]-1) + 1

trend_solar = scipy.stats.norm.rvs(
    size=SAMPLES, 
    loc=+0.01, 
    scale=0.07/NINETY_TO_ONESIGMA, 
    random_state=uncertainty_seed
)

scale_df = pd.DataFrame(
    data = scale,
    columns = ['co2','ch4','n2o','other_wmghg','o3','h2o_stratospheric','contrails','bc_on_snow','land_use','volcanic','solar']
)

In [ ]:
scale_df

In [ ]:
## put solar and volcanic here

## Aerosol forcing

In AR6, ERFari was based on emissions to forcing coefficients from Myhre et al (2013) https://acp.copernicus.org/articles/13/1853/2013/. At the time, I deemed there not sufficient evidence from CMIP6 AerChemMIP models or any other sources to update these. The uncertainty ranges from each precursor were expanded slightly compared to Myhre et al., in order to reproduce the overall ERFari uncertainty assessment (assumed that uncertainties in individual components are uncorrelated).

Following AR6 and a re-calibration of FaIR, I now use Bill Collins/Terje Bertnsen/Sara Blichner/Sophie Szopa's chapter 6 correspondences of emissions or concentrations to forcing.

ERFaci is based on fits to CMIP6 models from Smith et al. (2021) now updated to include 13 models and correct APRP code from Mark Zelinka.

Rescale both to the assessed forcings of -0.3 W/m2 for ERFari 2005-14 and -1.0 for ERFaci 2005-14.

In [ ]:
# these come from AR6 WG1
# source: https://github.com/sarambl/AR6_CH6_RCMIPFIGS/blob/master/ar6_ch6_rcmipfigs/data_out/fig6_12_ts15_historic_delta_GSAT/2019_ERF_est.csv
# they sum to -0.22 W/m2, for 2019
# Calculate a radiative efficiency for each species from CEDS and updated concentrations.
df_ari_emitted_mean = pd.read_csv('../data/ar6/table_mean_thornhill_collins_orignames.csv', index_col=0)
erfari_emitted = pd.Series(df_ari_emitted_mean['Aerosol'])
erfari_emitted.rename_axis(None, inplace=True)
erfari_emitted.rename({'HC': 'EESC', 'VOC': 'NMVOC'}, inplace=True)
erfari_emitted

In [ ]:
df_ari_emitted_std = pd.read_csv('../data/ar6/table_std_thornhill_collins_orignames.csv', index_col=0)
erfari_emitted_std = pd.Series(df_ari_emitted_std['Aerosol_sd'])
erfari_emitted_std.rename_axis(None, inplace=True)
erfari_emitted_std.rename({'HC': 'EESC', 'VOC': 'NMVOC'}, inplace=True)
erfari_emitted_std

In [ ]:
def calculate_eesc(
    concentration,
    fractional_release,
    fractional_release_cfc11,
    cl_atoms,
    br_atoms,
    br_cl_ratio=45,
):

    # EESC is in terms of CFC11-eq
    eesc_out = (
        cl_atoms * (concentration) * fractional_release / fractional_release_cfc11
        + br_cl_ratio
        * br_atoms
        * (concentration)
        * fractional_release
        / fractional_release_cfc11
    ) * fractional_release_cfc11
    return eesc_out


fractional_release = {
    "CFC-11": 0.47,
    "CFC-12": 0.23,
    "CFC-113": 0.29,
    "CFC-114": 0.12,
    "CFC-115": 0.04,
    "HCFC-22": 0.13,
    "HCFC-141b": 0.34,
    "HCFC-142b": 0.17,
    "CCl4": 0.56,
    "CHCl3": 0,
    "CH2Cl2": 0,
    "CH3Cl": 0.44,
    "CH3CCl3": 0.67,
    "CH3Br": 0.6,
    "Halon-1211": 0.62,
    "Halon-1301": 0.28,
    "Halon-2402": 0.65,
}

cl_atoms = {
    "CFC-11": 3,
    "CFC-12": 2,
    "CFC-113": 3,
    "CFC-114": 2,
    "CFC-115": 1,
    "HCFC-22": 1,
    "HCFC-141b": 2,
    "HCFC-142b": 1,
    "CCl4": 4,
    "CHCl3": 3,
    "CH2Cl2": 2,
    "CH3Cl": 1,
    "CH3CCl3": 3,
    "CH3Br": 0,
    "Halon-1211": 1,
    "Halon-1301": 0,
    "Halon-2402": 0,
}

br_atoms = {
    "CFC-11": 0,
    "CFC-12": 0,
    "CFC-113": 0,
    "CFC-114": 0,
    "CFC-115": 0,
    "HCFC-22": 0,
    "HCFC-141b": 0,
    "HCFC-142b": 0,
    "CCl4": 0,
    "CHCl3": 0,
    "CH2Cl2": 0,
    "CH3Cl": 0,
    "CH3CCl3": 0,
    "CH3Br": 1,
    "Halon-1211": 1,
    "Halon-1301": 1,
    "Halon-2402": 2,
}

hc_eesc = {}
total_eesc = np.zeros(273)
for species in cl_atoms:
    hc_eesc[species] = calculate_eesc(
        concentrations.loc[:, species],
        fractional_release[species],
        fractional_release["CFC-11"],
        cl_atoms[species],
        br_atoms[species],
    )
    total_eesc = total_eesc + hc_eesc[species]

total_eesc

In [ ]:
#total_eesc = total_eesc.to_frame('EESC')

In [ ]:
# erfari radiative efficiency per Mt or ppb or ppt
re = erfari_emitted / (emissions.loc[2019, :] - emissions.loc[1750, :])
re.dropna(inplace=True)

In [ ]:
re['CH4'] = erfari_emitted['CH4'] / (concentrations.loc[2019, 'CH4'] - concentrations.loc[1750, 'CH4'])
re['N2O'] = erfari_emitted['N2O'] / (concentrations.loc[2019, 'N2O'] - concentrations.loc[1750, 'N2O'])
re['EESC'] = erfari_emitted['EESC'] / (total_eesc.loc[2019] - total_eesc.loc[1750])

In [ ]:
re

In [ ]:
re_std = erfari_emitted_std / (emissions.loc[2019, :] - emissions.loc[1750, :])
re_std.dropna(inplace=True)
re_std['CH4'] = erfari_emitted_std['CH4'] / (concentrations.loc[2019, 'CH4'] - concentrations.loc[1750, 'CH4'])
re_std['N2O'] = erfari_emitted_std['N2O'] / (concentrations.loc[2019, 'N2O'] - concentrations.loc[1750, 'N2O'])
re_std['EESC'] = erfari_emitted_std['EESC'] / (total_eesc.loc[2019] - total_eesc.loc[1750])
re_std

In [ ]:
re.index

In [ ]:
erfari_best = pd.concat(
    (
        (re * emissions)[['BC', 'OC', 'SO2', 'NOx', 'NMVOC', 'NH3']] - (re * emissions.loc[1750, ['BC', 'OC', 'SO2', 'NOx', 'NMVOC', 'NH3']]),
        (re * concentrations)[['CH4', 'N2O']] - (re * concentrations.loc[1750, ['CH4', 'N2O']]),
        re['EESC'] * (total_eesc - total_eesc.loc[1750])
    ), axis=1
).dropna(axis=1).sum(axis=1)

In [ ]:
# 90% range of ERF uncertainty in 2019 from model estimates
np.sqrt((erfari_emitted_std**2).sum()) * NINETY_TO_ONESIGMA

In [ ]:
# 90% range of ERF uncertainty in 2005-2014 from model estimates
(erfari_best.loc[2005:2014].mean()/-0.22) * np.sqrt((erfari_emitted_std**2).sum()) * NINETY_TO_ONESIGMA

In [ ]:
# best estimate ERF in 2005-2014 from model estimates
erfari_best.loc[2005:2014].mean()

In [ ]:
# we need to map the -0.27 +/- 0.57 to -0.3 +/- 0.3 which is the IPCC AR6 assessment
best_scale = -0.3 / erfari_best.loc[2005:2014].mean()
unc_scale = 0.3 / ((erfari_best.loc[2005:2014].mean()/-0.22) * np.sqrt((erfari_emitted_std**2).sum()) * NINETY_TO_ONESIGMA)

In [ ]:
best_scale, unc_scale

In [ ]:
erfari_best * best_scale

In [ ]:
# convert to numpy for efficiency
erfari_re_samples = pd.DataFrame(
    scipy.stats.norm.rvs(
        re*best_scale, re_std*unc_scale, size=(SAMPLES, 9), random_state=3729329,
    ),
    columns = re.index
)[['BC', 'OC', 'SO2', 'NOx', 'NMVOC', 'NH3', 'CH4', 'N2O', 'EESC']]

In [ ]:
erfari_re_samples

In [ ]:
erfari_re_samples = erfari_re_samples.to_numpy()

In [ ]:
emnump = emissions.drop(columns=['CO']).to_numpy()

In [ ]:
erfari = np.zeros((273, SAMPLES))
for i in tqdm(range(SAMPLES)):
    erfari[:, i] = (
        (
            ((erfari_re_samples[i, :6] * emnump) - (erfari_re_samples[i, :6] * emnump[0, :])).sum(axis=1) + 
            ((erfari_re_samples[i, 6] * concentrations['CH4'].values) - (erfari_re_samples[i, 6] * concentrations.loc[1750, 'CH4'])) +
            ((erfari_re_samples[i, 7] * concentrations['N2O'].values) - (erfari_re_samples[i, 7] * concentrations.loc[1750, 'N2O'])) +
            (erfari_re_samples[i, 8] * (total_eesc.values - total_eesc.loc[1750]))
        )
    )

In [ ]:
pl.plot(erfari_best * best_scale)

In [ ]:
np.percentile(erfari[255:265, :].mean(axis=0), (5, 50, 95))

In [ ]:
df_aci_cal = pd.read_csv('../data/fair-calibrate-1.2.0/aerosol_cloud.csv', index_col=0)

In [ ]:
df_aci_cal

In [ ]:
beta_samp = df_aci_cal["aci_scale"]
n0_samp = df_aci_cal["Sulfur"]
n1_samp = df_aci_cal["BC"]
n2_samp = df_aci_cal["OC"]

In [ ]:
np.log(n0_samp)

In [ ]:
np.log(n1_samp)

In [ ]:
np.log(n2_samp)

In [ ]:
kde = scipy.stats.gaussian_kde([np.log(n0_samp), np.log(n1_samp), np.log(n2_samp)], bw_method=0.1)
aci_sample = kde.resample(size=SAMPLES * 1, seed=63648708)

In [ ]:
#aci_sample[0, aci_sample[0, :] > 0] = 0#np.nan
#aci_sample[1, aci_sample[1, :] > 0] = 0#np.nan
#aci_sample[2, aci_sample[2, :] > 0] = 0#np.nan
# mask = np.any(np.isnan(aci_sample), axis=0)
# aci_sample = aci_sample[:, ~mask]

In [ ]:
erfaci_sample = scipy.stats.norm.rvs(
    size=SAMPLES, loc=-1.0, scale=0.7/NINETY_TO_ONESIGMA, random_state=71271
)

In [ ]:
so2 = emissions['SO2'].values
bc = emissions['BC'].values
oc = emissions['OC'].values

In [ ]:
beta = np.zeros(SAMPLES)

In [ ]:
def aci_log(x, beta, n0, n1, n2):
    aci = beta * np.log(1 + x[0] * n0 + x[1] * n1 + x[2] * n2)
    return aci

In [ ]:
erfaci = np.zeros((273, SAMPLES))
for i in tqdm(range(SAMPLES), desc="aci samples"):
    ts2010 = np.mean(
        aci_log(
            [so2[255:265], bc[255:265], oc[255:265]],
            1.1,
            np.exp(aci_sample[0, i]),
            np.exp(aci_sample[1, i]),
            np.exp(aci_sample[2, i]),
        )
    )
    ts1750 = aci_log(
        [so2[0], bc[0], oc[0]],
        1.1,
        np.exp(aci_sample[0, i]),
        np.exp(aci_sample[1, i]),
        np.exp(aci_sample[2, i]),
    )
    erfaci[:, i] = (
       (
           aci_log(
               [so2, bc, oc],
               1.1,
               np.exp(aci_sample[0, i]),
               np.exp(aci_sample[1, i]),
               np.exp(aci_sample[2, i]),
           )
           - ts1750
       )
       / (ts2010 - ts1750)
       * (erfaci_sample[i])
    )
    beta[i] = erfaci_sample[i] / (ts2010 - ts1750)

In [ ]:
#pl.plot(erfaci);
#pl.ylim(-6, 0.2)

In [ ]:
(erfaci[255:265, :]).mean()

In [ ]:
np.median(erfaci[255:265, :].mean(axis=0))

## Contrail forcing

Based on Lee et al 2018 forcing of 0.0574. I recreated their original methods (see appendix A and supplementary data in their paper) and updated for newer IEA and ICAO data on fuel consumption with some assumptions about historical efficiency. 

In [ ]:
df_contrails = pd.read_csv('../data/contrails/contrails.csv', index_col=0)
df_contrails

## Land use forcing

Use cumulative land use CO2 emissions, scale to -0.2 W/m2 for 1750 to 2019. Demonstrate this is fairly close to an observational estimate (Ghimire et al. 2015)

In [ ]:
df_gcp = pd.read_csv('../data/gcp_emissions/gcp_2022.csv', index_col=0)

In [ ]:
df_gcp['AFOLU']

In [ ]:
f = interp1d(df_ghimire['year'], df_ghimire['flux'], kind='linear', fill_value='extrapolate', bounds_error=False)
lusf2019 = -0.20/(f(2019)-f(1750))
pl.plot(np.arange(1750, 2023), lusf2019*(f(np.arange(1750,2023))-f(1750)))
pl.plot(np.cumsum(df_gcp['AFOLU'])*-0.00087)

In [ ]:
lusf2019 = -0.20/(np.cumsum(df_gcp['AFOLU']).loc[2019] - df_gcp.loc[1750, 'AFOLU'])
lusf2019

In [ ]:
(np.cumsum(df_gcp['AFOLU']) - df_gcp.loc[1750, 'AFOLU'])*lusf2019

# BC on snow

linear with emissions, 2019 ERF = 0.08

In [ ]:
emissions.loc[2019,'BC']
(0.08*(emissions['BC']-emissions.loc[1750,'BC'])/(emissions.loc[2019,'BC']-emissions.loc[1750,'BC']))
pl.plot((0.08*(emissions['BC']-emissions.loc[1750,'BC'])/(emissions.loc[2019,'BC']-emissions.loc[1750,'BC'])))

## Greenhouse gas concentrations

Here, tropospheric and surface adjustments are only implemented for CO2, CH4, N2O, CFC11 and CFC12 to convert SARF to ERF. There's an argument to uplift ERF by 5% for other GHGs based on land surface warming, but the total forcing will be very small and no single-forcing studies exist. This was not done in AR6 chapter 7.

Radiative efficiencies for F-gases are from Hodnebrog et al. 2020 https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019RG000691.

In [ ]:
concentrations

In [ ]:
meinshausen2020

In [ ]:
concentrations.loc[2022].values.shape

In [ ]:
# radiative efficiencies
# source: Hodnebrog et al 2020 https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019RG000691
radeff = {
    'HFC-125':      0.23378,
    'HFC-134a':     0.16714,
    'HFC-143a':     0.168,
    'HFC-152a':     0.10174,
    'HFC-227ea':    0.27325,
    'HFC-23':       0.19111,
    'HFC-236fa':    0.25069,
    'HFC-245fa':    0.24498,
    'HFC-32':       0.11144,
    'HFC-365mfc':   0.22813,
    'HFC-43-10mee': 0.35731,
    'NF3':          0.20448,
    'C2F6':         0.26105,
    'C3F8':         0.26999,
    'n-C4F10':      0.36874,
    'n-C5F12':      0.4076,
    'n-C6F14':      0.44888,
    'i-C6F14':      0.44888,
    'C7F16':        0.50312,
    'C8F18':        0.55787,
    'CF4':          0.09859,
    'c-C4F8':       0.31392,
    'SF6':          0.56657,
    'SO2F2':        0.21074,
    'CCl4':         0.16616,
    'CFC-11':       0.25941,
    'CFC-112':      0.28192,
    'CFC-112a':     0.24564,
    'CFC-113':      0.30142,
    'CFC-113a':     0.24094, 
    'CFC-114':      0.31433,
    'CFC-114a':     0.29747,
    'CFC-115':      0.24625,
    'CFC-12':       0.31998,
    'CFC-13':       0.27752,
    'CH2Cl2':       0.02882,
    'CH3Br':        0.00432,
    'CH3CCl3':      0.06454,
    'CH3Cl':        0.00466,
    'CHCl3':        0.07357,
    'HCFC-124':     0.20721,
    'HCFC-133a':    0.14995,
    'HCFC-141b':    0.16065,
    'HCFC-142b':    0.19329,
    'HCFC-22':      0.21385,
    'HCFC-31':      0.068,
    'Halon-1202':   0,       # not in dataset
    'Halon-1211':   0.30014,
    'Halon-1301':   0.29943,
    'Halon-2402':   0.31169,
    'CO2':          0,       # different relationship
    'CH4':          0,       # different relationship
    'N2O':          0        # different relationship
}

radeff_array = np.ones(52) * np.nan
for igas, gas in enumerate(concentrations.columns):
    radeff_array[igas] = radeff[gas]

In [ ]:
np.where(concentrations.columns=='CFC-11')[0][0], np.where(concentrations.columns=='CFC-12')[0][0]

In [ ]:
adjustments = np.ones(52)
adjustments[0] = 1.05
adjustments[1] = 0.86
adjustments[2] = 1.07
adjustments[21] = 1.12
adjustments[22] = 1.13

In [ ]:
meinshausen2020(
    concentrations.loc[2019].values,
    concentrations.loc[1750].values,
    adjustments,
    radeff_array,
    [0],
    [1],
    [2],
    list(range(3,52))
)

In [ ]:
ghg_out = np.zeros((273, 52))
for i, year in enumerate(range(1750, 2023)):
    ghg_out[i, :] = meinshausen2020(
        concentrations.loc[year].values,
        concentrations.loc[1750].values,
        adjustments,
        radeff_array,
        [0],
        [1],
        [2],
        list(range(3,52))
    )
for igas, gas in enumerate(concentrations.columns):
    forcing[gas] = ghg_out[:, igas]

In [ ]:
pl.plot(ghg_out.sum(axis=1))
pl.plot(forcing['CO2'])

## Ozone

Reproduce the code from 070